In [1]:
import os, re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from torch import nn, optim
# from transformers import AutoTokenizer, AutoModel

In [3]:
train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv') 
test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv') 
train.head()

In [8]:
# train = pd.read_csv(r'D:\ds_ridwan\eval_student\summaries_train.csv') 
# test = pd.read_csv(r'D:\ds_ridwan\eval_student\summaries_test.csv') 
# train.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


In [3]:
prompt_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
prompt_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [6]:
prompt_train.prompt_question.iloc[1]

'In complete sentences, summarize the structure of the ancient Egyptian system of government. How were different social classes involved in this government? Cite evidence from the text.'

In [5]:
train.text.iloc[2]

'In Egypt, there were many occupations and social classes involved in day-to-day living. In many instances if you were at the bottom of the social ladder you could climb up, you didn\'t have to stay a peasant you could work to bring your status up. Everyone worshipped the gods Ra, Osiris, and Isis, but also they would worship their pharaohs like gods as well. Under the pharaohs were the priests, they had the responsibility to entertain or please the said god. The Chain of Command was placed to keep everyone in check, not one person could handle all the civilians and treasures without any aid. Like the tax collector, called a vizier like stated they were in charge of collecting the peoples\' tax. They were also one of the rare instances who were able to read and write, that\'s how they were granted "vizier" Also the soldiers did many things as they would fight in wars or "quelled domestic uprisings". They were in charge of getting the slaves, farmers, and peasants to build palaces or th

In [4]:
train.text.iloc[3]

'The highest class was Pharaohs these people were gods.Then the 2nd highest class was a gonvener.Chiefs minister were called a vizier as a supervisor. (par.6)'

## here, im not using prompt feature... 📌

# Cleaning

In [9]:
def expand_contractions(text):
    contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would",  
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is", 
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",  
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",  
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
    }
    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    
    return contractions_re.sub(lambda match: contractions_dict[match.group(0)], text)

In [10]:
def clean_text(text):
    text = text.lower()

    text = re.compile(r'<.*?>').sub(r'', text)

    text = re.sub(r'@\w+\s*', '', text)

    text = re.sub(r'#\w+', '', text)

    text = re.sub(r'http\S+|www\S+', '', text)

    text = text.replace(u'\xa0', ' ')

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text)
    
    text = expand_contractions(text)

    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\,+', ',', text)

    text = text.strip()

    return text

In [11]:
train['text'] = train['text'].apply(clean_text)

In [12]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
import torch
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader

# <font style='color:red'> ===|||||======|||||======|||||======|||||===

# Pipeline

In [13]:
X = train['text']
y = train[[
    'content',
    'wording'
]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
pipeline = Pipeline([
    ('vectorizer',TfidfVectorizer()),
    ('modeling',MultiOutputRegressor(LGBMRegressor()))
])

In [15]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('modeling', MultiOutputRegressor(estimator=LGBMRegressor()))])

In [16]:
y_pred = pipeline.predict(X_test)

In [17]:
y_pred

array([[ 0.47284903,  0.6289133 ],
       [-0.47468493, -0.13083022],
       [-0.60486934, -0.29940393],
       ...,
       [-0.1798596 ,  0.10465795],
       [ 0.93768958,  1.21847634],
       [-1.39286699, -1.02102206]])

In [18]:
# Calculate MSE for test data
content_mse = mean_squared_error(y_test['content'],y_pred[:,0])
wording_mse = mean_squared_error(y_test['wording'],y_pred[:,1])
print(f'Mean Squared Error for Content: {content_mse:.4f}')
print(f'Mean Squared Error for Wording: {wording_mse:.4f}')

Mean Squared Error for Content: 0.2499
Mean Squared Error for Wording: 0.4171


# <font style='color:red'> ===|||||======|||||======|||||======|||||===

## DeBERTa as Embedding

In [7]:
# Step 3: Load DeBERTa model and tokenizer
model_name = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
deberta_model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

## Prep data

In [10]:
class TextDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.float)
        }

## Build model arch

In [33]:
class CNNRegressor(nn.Module):
    def __init__(self, n_outputs, seq_len):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.seq_len = seq_len
        self.fc1 = nn.Linear(64 * self.seq_len // 4, 128)  # Adjusted based on pool and seq_len
        self.fc2 = nn.Linear(128, n_outputs)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = deberta_model(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden_state = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        x = last_hidden_state.permute(0, 2, 1)  # [batch_size, hidden_size, seq_len]
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train[['content', 'wording']], test_size=0.2, random_state=42)

## Dataset n Dataloader

In [35]:
train_dataset = TextDataset(X_train.tolist(), y_train.values.tolist(), tokenizer, max_len=512)
test_dataset = TextDataset(X_test.tolist(), y_test.values.tolist(), tokenizer, max_len=512)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [37]:
# configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deberta_model = deberta_model.to(device)
model = CNNRegressor(n_outputs=2, seq_len=512)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Train n Eval loop

In [38]:
def train_model(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

def eval_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
    return total_loss / len(data_loader)

In [39]:
n_epochs = 5
for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/{n_epochs}')
    train_loss = train_model(model, train_loader, criterion, optimizer, device)
    print(f'Train Loss: {train_loss:.4f}')
    eval_loss = eval_model(model, test_loader, criterion, device)
    print(f'Eval Loss: {eval_loss:.4f}')

Epoch 1/5


Training: 100%|██████████| 359/359 [01:21<00:00,  4.40it/s]


Train Loss: 0.4101


Evaluating: 100%|██████████| 90/90 [00:19<00:00,  4.50it/s]


Eval Loss: 0.3064
Epoch 2/5


...

Eval Loss: 0.2777
Epoch 5/5


Training: 100%|██████████| 359/359 [01:21<00:00,  4.42it/s]


Train Loss: 0.1534


Evaluating: 100%|██████████| 90/90 [00:20<00:00,  4.50it/s]

Eval Loss: 0.2743


In [42]:
def calculate_individual_mse(model, data_loader, device):
    model.eval()
    content_preds = []
    wording_preds = []
    content_true = []
    wording_true = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask)
            content_pred, wording_pred = outputs[:, 0], outputs[:, 1]
            content_true.extend(targets[:, 0].cpu().numpy())
            wording_true.extend(targets[:, 1].cpu().numpy())
            content_preds.extend(content_pred.cpu().numpy())
            wording_preds.extend(wording_pred.cpu().numpy())

    if len(content_preds) == 0 or len(wording_preds) == 0:
        return None, None

    content_mse = mean_squared_error(content_true, content_preds)
    wording_mse = mean_squared_error(wording_true, wording_preds)
    return content_mse, wording_mse

In [43]:
# Calculate MSE for test data
content_mse, wording_mse = calculate_individual_mse(model, test_loader, device)
print(f'Mean Squared Error for Content: {content_mse:.4f}')
print(f'Mean Squared Error for Wording: {wording_mse:.4f}')

Predicting: 100%|██████████| 90/90 [00:20<00:00,  4.49it/s]

Mean Squared Error for Content: 0.2046
Mean Squared Error for Wording: 0.3455
